# Generate training and prediction data from insitu and gridded datasets


## Load modules

In [ ]:
from datacube.utils.dask import start_local_dask

import sys
sys.path.append('/g/data/os22/chad_tmp/NEE_modelling/')
from collect_training_data import extract_ec_gridded_data

In [ ]:
client = start_local_dask(mem_safety_margin='2Gb')
client

## Collect training data from EC towers, and gridded data at pixel location of EC tower

This data is used for training the models.

'Yanco/2021_v1/L6/default/Yanco_L6_20130101_20210724_Monthly.nc', soil site

'ozflux/Collie/2021_v1/L5/default/Collie_L5_20170804_20191111.nc', COLLIE SITE AT Level 5

In [ ]:
suffixes = [
    'CumberlandPlain/2022_v1/L6/default/CumberlandPlain_L6_20140101_20220101_Monthly.nc',
    'Tumbarumba/2021_v1/L6/default/Tumbarumba_L6_20020107_20191231_Monthly.nc',
    'Whroo/2021_v1/L6/default/Whroo_L6_20111201_20210724_Monthly.nc',
    'WombatStateForest/2022_v1/L6/default/WombatStateForest_L6_20100120_20210529_Monthly.nc',
    'WallabyCreek/2022_v1/L6/default/WallabyCreek_L6_20050825_20130409_Monthly.nc',
    'RobsonCreek/2022_v1/L6/default/RobsonCreek_L6_20130801_20211218_Monthly.nc',
    'CapeTribulation/2022_v1/L6/default/CapeTribulation_L6_20100101_20181102_Monthly.nc',
    'AliceSpringsMulga/2022_v1/L6/default/AliceSpringsMulga_L6_20100903_20211231_Monthly.nc',
    'CalperumChowilla/2022_v1/L6/default/Calperum_L6_20100730_20220216_Monthly.nc',
    'DryRiver/2022_v1/L6/default/DryRiver_L6_20091025_20220218_Monthly.nc',
    'Litchfield/2021_v1/L6/default/Litchfield_L6_20150623_20210725_Monthly.nc',
    'Gingin/2021_v1/L6/default/Gingin_L6_20111013_20201231_Monthly.nc',
    'TiTreeEast/2022_v1/L6/default/TiTreeEast_L6_20120718_20220117_Monthly.nc',
    'SturtPlains/2021_v1/L6/default/SturtPlains_L6_20080828_20210724_Monthly.nc',
    'RiggsCreek/2022_v1/L6/default/RiggsCreek_L6_20110101_20170712_Monthly.nc',
    'DalyPasture/2022_v1/L6/default/DalyPasture_L6_20080101_20130908_Monthly.nc',
    'Otway/2021_v1/L6/default/Otway_L6_20070811_20110101_Monthly.nc',
]

i=0
sites = []
for suffix in suffixes:
    print(" {:02}/{:02}\r".format(i + 1, len(suffixes)), end="")
    xx = extract_ec_gridded_data(suffix)
    sites.append(xx)
    i+=1
    

## Collate gridded data for predictions

This data is used to predict on, results of this are stored here `/g/data/os22/chad_tmp/NEE_modelling/results/predicton_data/prediction_data_YYYY.nc`

In [1]:
import sys
sys.path.append('/g/data/os22/chad_tmp/NEE_modelling/')
from collect_prediction_data import collect_prediction_data

In [2]:
t1, t2='2002-10','2021'

In [3]:
ds = collect_prediction_data(time_start=t1, time_end=t2, verbose=True)
ds

   Extracting MODIS LAI
   Extracting MODIS LST
   Extracting MODIS fPAR
   Extracting dT
   Extracting SPEI
   Extracting Climate
   Cumulative rainfall
   Adding Landcover class
   Merge and create valid data mask
   Exporting netcdf


<xarray.Dataset>
Dimensions:      (time: 231, y: 681, x: 841)
Coordinates:
  * y            (y) float32 -10.0 -10.05 -10.1 -10.15 ... -43.9 -43.95 -44.0
  * x            (x) float32 112.0 112.1 112.1 112.2 ... 153.9 153.9 153.9 154.0
    spatial_ref  int32 4236
  * time         (time) datetime64[ns] 2002-10-15 2002-11-15 ... 2021-12-15
Data variables:
    lai          (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    LST          (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    Fpar         (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    LST-Tair     (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    spei         (y, x, time) float32 nan nan nan nan nan ... nan nan nan nan
    solar        (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    Ta           (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    VPD          (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    precip       (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    precip_cml   (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    IGBP_class   (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan